In [13]:
import pandas as pd
from pathlib import Path

# Folder where your CSVs live (use "." for current folder)
folder = Path(".")

files = sorted(folder.glob("post_info_*_diff.csv"))

# Read and combine
dfs = [pd.read_csv(f) for f in files]
combined = pd.concat(dfs, ignore_index=True)

# (Optional) add a year column extracted from filename
combined["year"] = [f.stem.split("_")[2] for f in files for _ in range(len(pd.read_csv(f)))]

# Save
combined.to_csv("post_info_2023_2026_diff_combined.csv", index=False)

print(f"Combined {len(files)} files into {len(combined):,} rows.")

Combined 4 files into 5,293 rows.


In [25]:
"""
TRAIN + SAVE ALL MODELS (HOLDOUT + RANDOM SPLIT)

What this script does:

A) HOLDOUT (season-based):
   - Train on 2023–2025, test on 2026
   - Tune GradientBoosting with GroupKFold (groups=season)
   - Evaluate all models on 2026
   - Save *every* fitted model as a joblib bundle (correct scaler + feature order)

B) RANDOM SPLIT (mixed years):
   - Train/test split across all 2023–2026 rows
   - Tune GradientBoosting with KFold on TRAIN only
   - Evaluate all models on TEST
   - Save *every* fitted model as a joblib bundle (correct scaler + feature order)

Bundles are saved in:
  models_holdout/*.joblib
  models_random/*.joblib

Each bundle contains:
  {
    "tag": "...",
    "model_name": "...",
    "model": fitted_estimator_or_pipeline,
    "features": [...],
    "numeric_cols": [...],
    "binary_cols": [...],
    "scaler": fitted_scaler_or_None,
    "metrics": {"MAE":..., "R2":...}   # on the chosen evaluation set
  }

Prediction later:
  - Load bundle with joblib.load(...)
  - Build X=df[bundle["features"]]
  - If bundle["scaler"] not None: scale numeric_cols
  - model.predict(X)
"""

import os
import joblib
import numpy as np
import pandas as pd

from sklearn.base import clone
from sklearn.model_selection import (
    train_test_split, GroupKFold, KFold, RandomizedSearchCV
)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score, make_scorer

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, HuberRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# ======================================================
# CONFIG
# ======================================================
TARGET = "point_spread"
RANDOM_STATE = 42
TEST_SIZE = 0.20

FEATURES_BASE = [
    "net_rating_diff",
    "efg_pct_diff",
    "luck_diff",
    "adj_off_eff_diff",
    "home_advantage",
    "rating_luck_interaction",
    "shooting_strength_interaction",
    "is_conference",
    "is_neutral",
]

HOLDOUT_TRAIN_FILES = [
    ("post_info_2023_diff.csv", 2023),
    ("post_info_2024_diff.csv", 2024),
    ("post_info_2025_diff.csv", 2025),
]
HOLDOUT_TEST_FILE = ("post_info_2026_diff.csv", 2026)

RANDOM_FILES = [
    "post_info_2023_diff.csv",
    "post_info_2024_diff.csv",
    "post_info_2025_diff.csv",
    "post_info_2026_diff.csv",
]

OUT_DIR_HOLDOUT = "models_holdout"
OUT_DIR_RANDOM  = "models_random"

os.makedirs(OUT_DIR_HOLDOUT, exist_ok=True)
os.makedirs(OUT_DIR_RANDOM,  exist_ok=True)


# ======================================================
# HELPERS
# ======================================================
def infer_cols(features):
    """
    Keep your original rule: don't scale flags, scale everything else.
    (Note: home_advantage is treated as "flag-like" in your code. We keep that.)
    """
    binary_cols = [c for c in ["is_conference", "is_neutral", "home_advantage"] if c in features]
    numeric_cols = [c for c in features if c not in binary_cols]
    return numeric_cols, binary_cols


def scale_numeric_only(X_train, X_test, numeric_cols):
    """
    Fit scaler on train numeric_cols; transform train/test numeric_cols.
    Returns: X_train_scaled, X_test_scaled, fitted_scaler
    """
    scaler = StandardScaler()
    Xtr = X_train.copy()
    Xte = X_test.copy()

    if numeric_cols:
        Xtr[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
        Xte[numeric_cols] = scaler.transform(X_test[numeric_cols])

    return Xtr, Xte, scaler


def fit_eval_and_save(
    model,
    model_name,
    X_train_raw,
    y_train,
    X_test_raw,
    y_test,
    features,
    numeric_cols,
    binary_cols,
    out_dir,
    tag,
    *,
    needs_external_scaler: bool
):
    """
    Fits model and evaluates on test set, then saves a joblib bundle.
    - If needs_external_scaler=True: scales numeric_cols using StandardScaler, stores scaler, fits model on scaled.
    - If False: fit model directly on raw X (e.g., RandomForest) OR a Pipeline that handles scaling internally.
    """
    m = clone(model)
    scaler = None

    if needs_external_scaler:
        Xtr, Xte, scaler = scale_numeric_only(X_train_raw, X_test_raw, numeric_cols)
        m.fit(Xtr, y_train)
        pred = m.predict(Xte)
    else:
        m.fit(X_train_raw, y_train)
        pred = m.predict(X_test_raw)

    mae = mean_absolute_error(y_test, pred)
    r2 = r2_score(y_test, pred)

    bundle = {
        "tag": tag,
        "model_name": model_name,
        "model": m,
        "features": features,          # exact order
        "numeric_cols": numeric_cols,
        "binary_cols": binary_cols,
        "scaler": scaler,              # None if not used
        "metrics": {"MAE": mae, "R2": r2},
    }

    out_path = os.path.join(out_dir, f"{model_name}.joblib")
    joblib.dump(bundle, out_path)
    print(f"✅ Saved {tag}: {out_path} | MAE={mae:.4f} R2={r2:.4f}")

    return mae, r2


def tune_gb_groupkfold(X_train, y_train, groups, random_state=42):
    """
    Tune GradientBoosting with GroupKFold.
    Uses a Pipeline with StandardScaler (harmless).
    """
    mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)

    gb_pipe = Pipeline([
        ("scaler", StandardScaler()),
        ("model", GradientBoostingRegressor(loss="absolute_error", random_state=random_state)),
    ])

    param_dist = {
        "model__n_estimators": [300, 600, 900, 1200],
        "model__learning_rate": [0.01, 0.03, 0.05, 0.08],
        "model__max_depth": [1, 2, 3],
        "model__subsample": [0.6, 0.8, 1.0],
        "model__min_samples_leaf": [1, 5, 15, 30],
    }

    cv = GroupKFold(n_splits=3)

    search = RandomizedSearchCV(
        gb_pipe,
        param_distributions=param_dist,
        n_iter=30,
        scoring=mae_scorer,
        cv=cv,
        random_state=random_state,
        n_jobs=-1,
    )

    search.fit(X_train, y_train, groups=groups)
    return search.best_estimator_, search.best_params_


def tune_gb_kfold(X_train, y_train, random_state=42):
    """
    Tune GradientBoosting with standard KFold on train only.
    Uses a Pipeline with StandardScaler (harmless).
    """
    mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)

    gb_pipe = Pipeline([
        ("scaler", StandardScaler()),
        ("model", GradientBoostingRegressor(loss="absolute_error", random_state=random_state)),
    ])

    param_dist = {
        "model__n_estimators": [300, 600, 900, 1200],
        "model__learning_rate": [0.01, 0.03, 0.05, 0.08],
        "model__max_depth": [1, 2, 3],
        "model__subsample": [0.6, 0.8, 1.0],
        "model__min_samples_leaf": [1, 5, 15, 30],
    }

    cv = KFold(n_splits=5, shuffle=True, random_state=random_state)

    search = RandomizedSearchCV(
        gb_pipe,
        param_distributions=param_dist,
        n_iter=30,
        scoring=mae_scorer,
        cv=cv,
        random_state=random_state,
        n_jobs=-1,
    )

    search.fit(X_train, y_train)
    return search.best_estimator_, search.best_params_


def build_models(best_gb_tuned):
    """
    Returns dict of model_name -> estimator/pipeline.
    """
    return {
        "LinearRegression": LinearRegression(),
        "Ridge": Ridge(alpha=1.0),
        "Lasso": Lasso(alpha=0.001, max_iter=10000),
        "ElasticNet": ElasticNet(alpha=0.001, l1_ratio=0.5, max_iter=10000),
        "HuberRegressor": HuberRegressor(epsilon=1.35, max_iter=2000),
        "RandomForest": RandomForestRegressor(
            n_estimators=500, random_state=RANDOM_STATE, min_samples_leaf=10
        ),
        "GradientBoosting_TUNED": best_gb_tuned,
    }


def needs_scaler(model_name):
    """
    Match your original behavior:
    - linear/huber: scaled numeric only (external scaler)
    - RandomForest: raw
    - GradientBoosting_TUNED: pipeline handles scaling
    """
    if model_name in ["RandomForest", "GradientBoosting_TUNED"]:
        return False
    return True


# ======================================================
# A) HOLDOUT: train 2023–2025, test 2026
# ======================================================
print("\n================ HOLDOUT (Train 2023–2025, Test 2026) ================\n")

# Load holdout train with season groups
train_parts = []
for f, yr in HOLDOUT_TRAIN_FILES:
    d = pd.read_csv(f)
    d["season"] = yr
    train_parts.append(d)

train_df = pd.concat(train_parts, ignore_index=True)
test_df = pd.read_csv(HOLDOUT_TEST_FILE[0])

# Ensure features exist in BOTH
features_holdout = [c for c in FEATURES_BASE if c in train_df.columns and c in test_df.columns]

# Drop missing
train_df = train_df.dropna(subset=features_holdout + [TARGET]).copy()
test_df  = test_df.dropna(subset=features_holdout + [TARGET]).copy()

X_train = train_df[features_holdout].copy()
y_train = train_df[TARGET].copy()
groups  = train_df["season"].copy()

X_test = test_df[features_holdout].copy()
y_test = test_df[TARGET].copy()

numeric_cols, binary_cols = infer_cols(features_holdout)

# Tune GB with GroupKFold
best_gb, best_params = tune_gb_groupkfold(X_train, y_train, groups, random_state=RANDOM_STATE)
print("Best params (GB tuned, GroupKFold):", best_params)

# Evaluate and save all models
models_holdout = build_models(best_gb)
holdout_results = []

for name, model in models_holdout.items():
    mae, r2 = fit_eval_and_save(
        model=model,
        model_name=f"holdout_{name}",
        X_train_raw=X_train,
        y_train=y_train,
        X_test_raw=X_test,
        y_test=y_test,
        features=features_holdout,
        numeric_cols=numeric_cols,
        binary_cols=binary_cols,
        out_dir=OUT_DIR_HOLDOUT,
        tag="HOLDOUT_train_2023_2025",
        needs_external_scaler=needs_scaler(name),
    )
    holdout_results.append({"Model": name, "MAE": mae, "R2": r2})

holdout_res_df = pd.DataFrame(holdout_results).sort_values("MAE").reset_index(drop=True)
print("\n🏀 Holdout Performance Ranking:")
print(holdout_res_df.to_string(index=False))
print("\n🌟 Best by MAE (holdout):", holdout_res_df.iloc[0]["Model"])


# ======================================================
# B) RANDOM SPLIT: mix all years 2023–2026
# ======================================================
print("\n================ RANDOM SPLIT (Mixed 2023–2026) ================\n")

df_all = pd.concat([pd.read_csv(f) for f in RANDOM_FILES], ignore_index=True)

features_random = [c for c in FEATURES_BASE if c in df_all.columns]
df_all = df_all.dropna(subset=features_random + [TARGET]).copy()

X = df_all[features_random].copy()
y = df_all[TARGET].copy()

numeric_cols_r, binary_cols_r = infer_cols(features_random)

X_tr, X_te, y_tr, y_te = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE
)

print(f"✅ Total rows: {len(df_all):,}")
print(f"✅ Train rows: {len(X_tr):,} | Test rows: {len(X_te):,}")
print(f"✅ Features ({len(features_random)}): {features_random}")
print(f"✅ Numeric scaled: {numeric_cols_r}")
print(f"✅ Left unscaled (flags): {binary_cols_r}")

# Tune GB with KFold on TRAIN only
best_gb_r, best_params_r = tune_gb_kfold(X_tr, y_tr, random_state=RANDOM_STATE)
print("Best params (GB tuned, KFold):", best_params_r)

models_random = build_models(best_gb_r)
random_results = []

for name, model in models_random.items():
    mae, r2 = fit_eval_and_save(
        model=model,
        model_name=f"random_{name}",
        X_train_raw=X_tr,
        y_train=y_tr,
        X_test_raw=X_te,
        y_test=y_te,
        features=features_random,
        numeric_cols=numeric_cols_r,
        binary_cols=binary_cols_r,
        out_dir=OUT_DIR_RANDOM,
        tag="RANDOMSPLIT_mixed_2023_2026",
        needs_external_scaler=needs_scaler(name),
    )
    random_results.append({"Model": name, "MAE": mae, "R2": r2})

random_res_df = pd.DataFrame(random_results).sort_values("MAE").reset_index(drop=True)
print("\n🏀 Random-Split Performance Ranking:")
print(random_res_df.to_string(index=False))
print("\n🌟 Best by MAE (random):", random_res_df.iloc[0]["Model"])


print("\n✅ Done. All models saved into:")
print(f" - {OUT_DIR_HOLDOUT}/ (holdout bundles)")
print(f" - {OUT_DIR_RANDOM}/  (random-split bundles)")


# ======================================================
# OPTIONAL: HOW TO PREDICT FROM A SAVED BUNDLE (example)
# ======================================================
# import joblib, pandas as pd
#
# bundle = joblib.load("models_holdout/holdout_GradientBoosting_TUNED.joblib")
# df_future = pd.read_csv("post_info_2026_diff.csv")  # upcoming games feature file
#
# feats = bundle["features"]
# Xf = df_future[feats].apply(pd.to_numeric, errors="coerce").fillna(0)
#
# if bundle["scaler"] is not None and bundle["numeric_cols"]:
#     Xf[bundle["numeric_cols"]] = bundle["scaler"].transform(Xf[bundle["numeric_cols"]])
#
# preds = bundle["model"].predict(Xf)
# print(preds[:10])


================ HOLDOUT (Train 2023–2025, Test 2026) ================

Best params (GB tuned, GroupKFold): {'model__subsample': 0.8, 'model__n_estimators': 300, 'model__min_samples_leaf': 15, 'model__max_depth': 3, 'model__learning_rate': 0.03}
✅ Saved HOLDOUT_train_2023_2025: models_holdout/holdout_LinearRegression.joblib | MAE=8.3593 R2=0.7165
✅ Saved HOLDOUT_train_2023_2025: models_holdout/holdout_Ridge.joblib | MAE=8.3597 R2=0.7165
✅ Saved HOLDOUT_train_2023_2025: models_holdout/holdout_Lasso.joblib | MAE=8.3591 R2=0.7165
✅ Saved HOLDOUT_train_2023_2025: models_holdout/holdout_ElasticNet.joblib | MAE=8.3601 R2=0.7164
✅ Saved HOLDOUT_train_2023_2025: models_holdout/holdout_HuberRegressor.joblib | MAE=8.3740 R2=0.7147
✅ Saved HOLDOUT_train_2023_2025: models_holdout/holdout_RandomForest.joblib | MAE=8.5661 R2=0.6993
✅ Saved HOLDOUT_train_2023_2025: models_holdout/holdout_GradientBoosting_TUNED.joblib | MAE=8.5742 R2=0.6952

🏀 Holdout Performance Ranking:
                 Model      

In [26]:
"""
TRAIN + SAVE ALL MODELS (HOLDOUT + RANDOM SPLIT) + CALIBRATE CONFORMAL PIs
AND (OPTIONAL) GENERATE PREDICTIONS + INTERVALS FOR 2026 FUTURE GAMES.

What this script does:

A) HOLDOUT (season-based):
   - Train on 2023–2025, test on 2026
   - Tune GradientBoosting with GroupKFold (groups=season)
   - Evaluate all models on 2026
   - Save *every* fitted model as a joblib bundle (correct scaler + feature order)
   - Calibrate conformal prediction interval half-width q on the same evaluation set

B) RANDOM SPLIT (mixed years):
   - Train/test split across all 2023–2026 rows
   - Tune GradientBoosting with KFold on TRAIN only
   - Evaluate all models on TEST
   - Save *every* fitted model as a joblib bundle (correct scaler + feature order)
   - Calibrate conformal prediction interval half-width q on the same evaluation set

C) OPTIONAL: Produce a prediction CSV for upcoming games (no y needed)
   - Reads FUTURE_FEATURE_FILE
   - Adds pred + ci_lb/ci_ub columns for every saved model
   - Writes post_2026_predictions_with_intervals.csv

Prediction Interval Method:
  Split conformal (symmetric):
    PI = pred ± q
  where q is a conservative (1 - alpha) quantile of |residual| on evaluation set.
  Default alpha=0.30 => target coverage 70%.

Bundles are saved in:
  models_holdout/*.joblib
  models_random/*.joblib

Each bundle contains:
  {
    "tag": "...",
    "model_name": "...",
    "model": fitted_estimator_or_pipeline,
    "features": [...],
    "numeric_cols": [...],
    "binary_cols": [...],
    "scaler": fitted_scaler_or_None,
    "metrics": {"MAE":..., "R2":...},
    "pi_method": "split_conformal_abs_residual",
    "pi_alpha": 0.30,
    "pi_halfwidth_q": ...,
    "pi_coverage_eval": ...,
    "pi_avg_width_eval": ...
  }
"""

import os
import joblib
import numpy as np
import pandas as pd

from sklearn.base import clone
from sklearn.model_selection import (
    train_test_split, GroupKFold, KFold, RandomizedSearchCV
)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score, make_scorer

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, HuberRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor


# ======================================================
# CONFIG
# ======================================================
TARGET = "point_spread"
RANDOM_STATE = 42
TEST_SIZE = 0.20

# Prediction interval target coverage = 1 - alpha
PI_ALPHA = 0.30  # 70% coverage target

FEATURES_BASE = [
    "net_rating_diff",
    "efg_pct_diff",
    "luck_diff",
    "adj_off_eff_diff",
    "home_advantage",
    "rating_luck_interaction",
    "shooting_strength_interaction",
    "is_conference",
    "is_neutral",
]

HOLDOUT_TRAIN_FILES = [
    ("post_info_2023_diff.csv", 2023),
    ("post_info_2024_diff.csv", 2024),
    ("post_info_2025_diff.csv", 2025),
]
HOLDOUT_TEST_FILE = ("post_info_2026_diff.csv", 2026)

RANDOM_FILES = [
    "post_info_2023_diff.csv",
    "post_info_2024_diff.csv",
    "post_info_2025_diff.csv",
    "post_info_2026_diff.csv",
]

OUT_DIR_HOLDOUT = "models_holdout"
OUT_DIR_RANDOM  = "models_random"

# OPTIONAL: produce predictions+intervals for an upcoming-games feature file.
# Set to None to skip. This file MUST contain the engineered feature columns.
FUTURE_FEATURE_FILE = "post_info_2026_diff.csv"   # change if needed
FUTURE_OUT_FILE = "post_2026_predictions_with_intervals.csv"

os.makedirs(OUT_DIR_HOLDOUT, exist_ok=True)
os.makedirs(OUT_DIR_RANDOM,  exist_ok=True)


# ======================================================
# HELPERS
# ======================================================
def infer_cols(features):
    """
    Keep your original rule: don't scale flags, scale everything else.
    (home_advantage treated as flag-like)
    """
    binary_cols = [c for c in ["is_conference", "is_neutral", "home_advantage"] if c in features]
    numeric_cols = [c for c in features if c not in binary_cols]
    return numeric_cols, binary_cols


def scale_numeric_only(X_train, X_test, numeric_cols):
    """
    Fit scaler on train numeric_cols; transform train/test numeric_cols.
    Returns: X_train_scaled, X_test_scaled, fitted_scaler
    """
    scaler = StandardScaler()
    Xtr = X_train.copy()
    Xte = X_test.copy()

    if numeric_cols:
        Xtr.loc[:, numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
        Xte.loc[:, numeric_cols] = scaler.transform(X_test[numeric_cols])

    return Xtr, Xte, scaler


def conformal_q(y_true, y_pred, alpha=0.30):
    """
    Conservative split-conformal quantile for symmetric PI: pred ± q.
    q is computed from absolute residuals.
    """
    abs_err = np.abs(np.asarray(y_true) - np.asarray(y_pred))
    n = len(abs_err)
    if n == 0:
        return np.nan

    # Conservative conformal quantile:
    # use ceil((n+1)*(1-alpha))/n and 'higher' method to avoid undercoverage
    q_level = np.ceil((n + 1) * (1 - alpha)) / n
    q_level = min(max(q_level, 0.0), 1.0)

    try:
        q = np.quantile(abs_err, q_level, method="higher")
    except TypeError:
        # older numpy fallback
        q = np.quantile(abs_err, q_level, interpolation="higher")

    return float(q)


def eval_pi(y_true, y_pred, q):
    """
    Evaluate coverage and avg width for symmetric PI pred ± q
    """
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    lb = y_pred - q
    ub = y_pred + q
    coverage = float(np.mean((y_true >= lb) & (y_true <= ub)))
    avg_width = float(np.mean(ub - lb))
    return coverage, avg_width


def fit_eval_and_save(
    model,
    model_name,
    X_train_raw,
    y_train,
    X_test_raw,
    y_test,
    features,
    numeric_cols,
    binary_cols,
    out_dir,
    tag,
    *,
    needs_external_scaler: bool,
    pi_alpha: float = 0.30,
):
    """
    Fits model and evaluates on test set, then saves a joblib bundle.
    Also calibrates conformal PI half-width q on the same evaluation set.
    """
    m = clone(model)
    scaler = None

    if needs_external_scaler:
        Xtr, Xte, scaler = scale_numeric_only(X_train_raw, X_test_raw, numeric_cols)
        m.fit(Xtr, y_train)
        pred = m.predict(Xte)
    else:
        m.fit(X_train_raw, y_train)
        pred = m.predict(X_test_raw)

    mae = mean_absolute_error(y_test, pred)
    r2 = r2_score(y_test, pred)

    # ---- conformal PI calibration on eval set ----
    q = conformal_q(y_test, pred, alpha=pi_alpha)
    cov, avg_w = eval_pi(y_test, pred, q)

    bundle = {
        "tag": tag,
        "model_name": model_name,
        "model": m,
        "features": features,          # exact order
        "numeric_cols": numeric_cols,
        "binary_cols": binary_cols,
        "scaler": scaler,              # None if not used
        "metrics": {"MAE": float(mae), "R2": float(r2)},

        "pi_method": "split_conformal_abs_residual",
        "pi_alpha": float(pi_alpha),
        "pi_target_coverage": float(1 - pi_alpha),
        "pi_halfwidth_q": float(q),
        "pi_coverage_eval": float(cov),
        "pi_avg_width_eval": float(avg_w),
    }

    out_path = os.path.join(out_dir, f"{model_name}.joblib")
    joblib.dump(bundle, out_path)
    print(
        f"✅ Saved {tag}: {out_path} | MAE={mae:.4f} R2={r2:.4f} "
        f"| PI q={q:.3f} cov={cov:.3%} avgW={avg_w:.3f}"
    )
    return mae, r2


def tune_gb_groupkfold(X_train, y_train, groups, random_state=42):
    """
    Tune GradientBoosting with GroupKFold (season groups).
    Uses a Pipeline with StandardScaler (harmless).
    """
    mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)

    gb_pipe = Pipeline([
        ("scaler", StandardScaler()),
        ("model", GradientBoostingRegressor(loss="absolute_error", random_state=random_state)),
    ])

    param_dist = {
        "model__n_estimators": [300, 600, 900, 1200],
        "model__learning_rate": [0.01, 0.03, 0.05, 0.08],
        "model__max_depth": [1, 2, 3],
        "model__subsample": [0.6, 0.8, 1.0],
        "model__min_samples_leaf": [1, 5, 15, 30],
    }

    cv = GroupKFold(n_splits=3)

    search = RandomizedSearchCV(
        gb_pipe,
        param_distributions=param_dist,
        n_iter=30,
        scoring=mae_scorer,
        cv=cv,
        random_state=random_state,
        n_jobs=-1,
    )

    search.fit(X_train, y_train, groups=groups)
    return search.best_estimator_, search.best_params_


def tune_gb_kfold(X_train, y_train, random_state=42):
    """
    Tune GradientBoosting with KFold on train only.
    Uses a Pipeline with StandardScaler (harmless).
    """
    mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)

    gb_pipe = Pipeline([
        ("scaler", StandardScaler()),
        ("model", GradientBoostingRegressor(loss="absolute_error", random_state=random_state)),
    ])

    param_dist = {
        "model__n_estimators": [300, 600, 900, 1200],
        "model__learning_rate": [0.01, 0.03, 0.05, 0.08],
        "model__max_depth": [1, 2, 3],
        "model__subsample": [0.6, 0.8, 1.0],
        "model__min_samples_leaf": [1, 5, 15, 30],
    }

    cv = KFold(n_splits=5, shuffle=True, random_state=random_state)

    search = RandomizedSearchCV(
        gb_pipe,
        param_distributions=param_dist,
        n_iter=30,
        scoring=mae_scorer,
        cv=cv,
        random_state=random_state,
        n_jobs=-1,
    )

    search.fit(X_train, y_train)
    return search.best_estimator_, search.best_params_


def build_models(best_gb_tuned):
    """
    Returns dict of model_name -> estimator/pipeline.
    """
    return {
        "LinearRegression": LinearRegression(),
        "Ridge": Ridge(alpha=1.0),
        "Lasso": Lasso(alpha=0.001, max_iter=10000),
        "ElasticNet": ElasticNet(alpha=0.001, l1_ratio=0.5, max_iter=10000),
        "HuberRegressor": HuberRegressor(epsilon=1.35, max_iter=2000),
        "RandomForest": RandomForestRegressor(
            n_estimators=500, random_state=RANDOM_STATE, min_samples_leaf=10
        ),
        "GradientBoosting_TUNED": best_gb_tuned,  # Pipeline
    }


def needs_scaler(model_name):
    """
    - linear/huber: scale numeric only (external scaler)
    - RandomForest: raw
    - GradientBoosting_TUNED: pipeline handles scaling
    """
    if model_name in ["RandomForest", "GradientBoosting_TUNED"]:
        return False
    return True


def bundle_predict_ci(bundle, df_features: pd.DataFrame):
    """
    Predict point spread + conformal PI using a saved bundle.
    Returns: pred, ci_lb, ci_ub (numpy arrays)
    """
    feats = bundle["features"]
    X = df_features[feats].copy()
    X = X.apply(pd.to_numeric, errors="coerce").fillna(0)

    scaler = bundle.get("scaler", None)
    numeric_cols = bundle.get("numeric_cols", [])
    if scaler is not None and numeric_cols:
        X.loc[:, numeric_cols] = scaler.transform(X[numeric_cols])

    pred = bundle["model"].predict(X)

    q = bundle.get("pi_halfwidth_q", np.nan)
    lb = pred - q
    ub = pred + q
    return pred, lb, ub


# ======================================================
# A) HOLDOUT: train 2023–2025, test 2026
# ======================================================
print("\n================ HOLDOUT (Train 2023–2025, Test 2026) ================\n")

# Load holdout train with season groups
train_parts = []
for f, yr in HOLDOUT_TRAIN_FILES:
    d = pd.read_csv(f)
    d["season"] = yr
    train_parts.append(d)

train_df = pd.concat(train_parts, ignore_index=True)
test_df = pd.read_csv(HOLDOUT_TEST_FILE[0])

# Ensure features exist in BOTH
features_holdout = [c for c in FEATURES_BASE if c in train_df.columns and c in test_df.columns]

# Drop missing
train_df = train_df.dropna(subset=features_holdout + [TARGET]).copy()
test_df  = test_df.dropna(subset=features_holdout + [TARGET]).copy()

X_train = train_df[features_holdout].copy()
y_train = train_df[TARGET].copy()
groups  = train_df["season"].copy()

X_test = test_df[features_holdout].copy()
y_test = test_df[TARGET].copy()

numeric_cols, binary_cols = infer_cols(features_holdout)

# Tune GB with GroupKFold
best_gb, best_params = tune_gb_groupkfold(X_train, y_train, groups, random_state=RANDOM_STATE)
print("Best params (GB tuned, GroupKFold):", best_params)

# Evaluate and save all models
models_holdout = build_models(best_gb)
holdout_results = []

for name, model in models_holdout.items():
    mae, r2 = fit_eval_and_save(
        model=model,
        model_name=f"holdout_{name}",
        X_train_raw=X_train,
        y_train=y_train,
        X_test_raw=X_test,
        y_test=y_test,
        features=features_holdout,
        numeric_cols=numeric_cols,
        binary_cols=binary_cols,
        out_dir=OUT_DIR_HOLDOUT,
        tag="HOLDOUT_train_2023_2025",
        needs_external_scaler=needs_scaler(name),
        pi_alpha=PI_ALPHA,
    )
    holdout_results.append({"Model": name, "MAE": mae, "R2": r2})

holdout_res_df = pd.DataFrame(holdout_results).sort_values("MAE").reset_index(drop=True)
print("\n🏀 Holdout Performance Ranking:")
print(holdout_res_df.to_string(index=False))
print("\n🌟 Best by MAE (holdout):", holdout_res_df.iloc[0]["Model"])


# ======================================================
# B) RANDOM SPLIT: mix all years 2023–2026
# ======================================================
print("\n================ RANDOM SPLIT (Mixed 2023–2026) ================\n")

df_all = pd.concat([pd.read_csv(f) for f in RANDOM_FILES], ignore_index=True)

features_random = [c for c in FEATURES_BASE if c in df_all.columns]
df_all = df_all.dropna(subset=features_random + [TARGET]).copy()

X = df_all[features_random].copy()
y = df_all[TARGET].copy()

numeric_cols_r, binary_cols_r = infer_cols(features_random)

X_tr, X_te, y_tr, y_te = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE
)

print(f"✅ Total rows: {len(df_all):,}")
print(f"✅ Train rows: {len(X_tr):,} | Test rows: {len(X_te):,}")
print(f"✅ Features ({len(features_random)}): {features_random}")
print(f"✅ Numeric scaled: {numeric_cols_r}")
print(f"✅ Left unscaled (flags): {binary_cols_r}")

# Tune GB with KFold on TRAIN only
best_gb_r, best_params_r = tune_gb_kfold(X_tr, y_tr, random_state=RANDOM_STATE)
print("Best params (GB tuned, KFold):", best_params_r)

models_random = build_models(best_gb_r)
random_results = []

for name, model in models_random.items():
    mae, r2 = fit_eval_and_save(
        model=model,
        model_name=f"random_{name}",
        X_train_raw=X_tr,
        y_train=y_tr,
        X_test_raw=X_te,
        y_test=y_te,
        features=features_random,
        numeric_cols=numeric_cols_r,
        binary_cols=binary_cols_r,
        out_dir=OUT_DIR_RANDOM,
        tag="RANDOMSPLIT_mixed_2023_2026",
        needs_external_scaler=needs_scaler(name),
        pi_alpha=PI_ALPHA,
    )
    random_results.append({"Model": name, "MAE": mae, "R2": r2})

random_res_df = pd.DataFrame(random_results).sort_values("MAE").reset_index(drop=True)
print("\n🏀 Random-Split Performance Ranking:")
print(random_res_df.to_string(index=False))
print("\n🌟 Best by MAE (random):", random_res_df.iloc[0]["Model"])


print("\n✅ Done. All models saved into:")
print(f" - {OUT_DIR_HOLDOUT}/ (holdout bundles)")
print(f" - {OUT_DIR_RANDOM}/  (random-split bundles)")


# ======================================================
# C) OPTIONAL: PREDICT FUTURE GAMES + INTERVALS
# ======================================================
if FUTURE_FEATURE_FILE is not None and os.path.exists(FUTURE_FEATURE_FILE):
    print("\n================ PREDICT FUTURE GAMES + INTERVALS ================\n")
    future_df = pd.read_csv(FUTURE_FEATURE_FILE)

    # You usually want to preserve order + teams
    cols_keep = [c for c in ["home_team", "away_team"] if c in future_df.columns]
    out = future_df[cols_keep].copy() if cols_keep else pd.DataFrame(index=future_df.index)

    # Predict for every saved model bundle
    def add_preds_from_dir(model_dir, prefix):
        for fn in sorted(os.listdir(model_dir)):
            if not fn.endswith(".joblib"):
                continue
            path = os.path.join(model_dir, fn)
            bundle = joblib.load(path)

            # Example bundle["model_name"] = "holdout_Lasso"
            name_clean = bundle["model_name"]
            if name_clean.startswith(prefix + "_"):
                name_clean = name_clean[len(prefix) + 1:]

            pred, lb, ub = bundle_predict_ci(bundle, future_df)
            out[f"prediction_spread_{prefix}_{name_clean}"] = pred
            out[f"ci_lb_{prefix}_{name_clean}"] = lb
            out[f"ci_ub_{prefix}_{name_clean}"] = ub

    add_preds_from_dir(OUT_DIR_HOLDOUT, "holdout")
    add_preds_from_dir(OUT_DIR_RANDOM,  "random")

    # Optional averages (point preds)
    holdout_pred_cols = [c for c in out.columns if c.startswith("prediction_spread_holdout_")]
    random_pred_cols  = [c for c in out.columns if c.startswith("prediction_spread_random_")]

    out["avg_holdout"] = out[holdout_pred_cols].mean(axis=1) if holdout_pred_cols else np.nan
    out["avg_random"]  = out[random_pred_cols].mean(axis=1)  if random_pred_cols  else np.nan
    out["avg_all"]     = out[holdout_pred_cols + random_pred_cols].mean(axis=1) if (holdout_pred_cols or random_pred_cols) else np.nan

    out.to_csv(FUTURE_OUT_FILE, index=False)
    print(f"✅ Saved predictions + intervals → {FUTURE_OUT_FILE}")
else:
    print("\n(Skipped future predictions: FUTURE_FEATURE_FILE is None or not found.)")


================ HOLDOUT (Train 2023–2025, Test 2026) ================

Best params (GB tuned, GroupKFold): {'model__subsample': 0.8, 'model__n_estimators': 300, 'model__min_samples_leaf': 15, 'model__max_depth': 3, 'model__learning_rate': 0.03}
✅ Saved HOLDOUT_train_2023_2025: models_holdout/holdout_LinearRegression.joblib | MAE=8.3593 R2=0.7165 | PI q=10.735 cov=70.062% avgW=21.470
✅ Saved HOLDOUT_train_2023_2025: models_holdout/holdout_Ridge.joblib | MAE=8.3597 R2=0.7165 | PI q=10.739 cov=70.062% avgW=21.478
✅ Saved HOLDOUT_train_2023_2025: models_holdout/holdout_Lasso.joblib | MAE=8.3591 R2=0.7165 | PI q=10.740 cov=70.062% avgW=21.480
✅ Saved HOLDOUT_train_2023_2025: models_holdout/holdout_ElasticNet.joblib | MAE=8.3601 R2=0.7164 | PI q=10.731 cov=70.062% avgW=21.462
✅ Saved HOLDOUT_train_2023_2025: models_holdout/holdout_HuberRegressor.joblib | MAE=8.3740 R2=0.7147 | PI q=10.732 cov=70.062% avgW=21.463
✅ Saved HOLDOUT_train_2023_2025: models_holdout/holdout_RandomForest.joblib | 